In [3]:
import pyspark
import random
sc = pyspark.SparkContext.getOrCreate()

In [4]:
import pandas as pd

from pyspark.sql import SQLContext, SparkSession
sqlContext = SQLContext.getOrCreate(sc)
sparkSession = SparkSession.builder.master('local').appName("Book advices").getOrCreate()

IllegalArgumentException: "Error while instantiating 'org.apache.spark.sql.hive.HiveSessionStateBuilder':"

In [ ]:
books = sparkSession.read.csv('./BX-Books.csv', sep=';', header=True)
users = sparkSession.read.csv('./BX-Users.csv', sep=';', header=True)
books_ratings = sparkSession.read.csv('./BX-Book-Ratings.csv', sep=';', header=True)

In [ ]:
books_ratings.show(5)

In [ ]:
type(books)

In [ ]:
pd.DataFrame(books.take(5))

In [ ]:
top_rated_books = books_ratings.groupBy('ISBN').count().orderBy('count', ascending=False)
top_rated_books.show(5)

In [ ]:
pd.DataFrame(books.filter('ISBN in ("0971880107","0316666343", "0385504209", "0060928336", "0312195516")').take(5))

In [ ]:
from pyspark.sql import functions as F

average_rating = books_ratings.groupBy('ISBN').agg(F.mean('book-rating'), F.count('book-rating')).orderBy('count(book-rating)', ascending=False).show(5)

In [ ]:
user_activity = books_ratings.groupBy('User-ID').count().orderBy('count', ascending=False)
user_activity.filter("count > 200").show(5)